In [1]:
import numpy as np
import pickle as pkl
import music21 as m21
from IPython.display import Image
from tqdm import tqdm_notebook
import time

In [2]:
import tensorflow as tf

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, TimeDistributed, Reshape, Lambda
from keras.layers import LSTM
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
allBach = m21.corpus.search('bach')
s1 = allBach[1].parse()

In [4]:
s1.show('text')

{0.0} <music21.text.TextBox "BWV 1.6  W...">
{0.0} <music21.text.TextBox "Harmonized...">
{0.0} <music21.text.TextBox "PDF ©2004 ...">
{0.0} <music21.metadata.Metadata object at 0x7f20e7b2a668>
{0.0} <music21.stream.Part Horn 2>
    {0.0} <music21.instrument.Instrument P1: Horn 2: Instrument 7>
    {0.0} <music21.stream.Measure 0 offset=0.0>
        {0.0} <music21.layout.PageLayout>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.key.Key of F major>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Note F>
    {1.0} <music21.stream.Measure 1 offset=1.0>
        {0.0} <music21.note.Note G>
        {0.5} <music21.note.Note C>
        {1.0} <music21.note.Note F>
        {1.5} <music21.note.Note F>
        {2.0} <music21.note.Note A>
        {2.5} <music21.note.Note F>
        {3.0} <music21.note.Note A>
        {3.5} <music21.note.Note C>
    {5.0} <music21.stream.Measure 2 offset=5.0>
        {0.0} <music21.note.Note F>
        {0.25} <music21.not

- if bar is last and note durations do not sum to the time signature value, extend final note to fill space...

In [30]:
# get measure division

# rhythms = {}

for part in s1.parts:
    denominator = 4
    numerator = 4
    print('NEW PART')
    print(part.partName)
    count = 0 
    for m in part.getElementsByClass('Measure'):
        ts = m.getElementsByClass('TimeSignature')
        if ts:
            denominator = ts[0].denominator
            numerator = ts[0].numerator
        
        print(count, m.analyze('key').tonic, end=' ')
        
        rw = []
#         n = m.notesAndRests
        
        for n in m.notesAndRests:
            if sum(rw) >= numerator/denominator:
                print(rw, end=' ')
                rw = []
            
#             rw.append(n.duration.quarterLength)
        
        print(rw)
        
#         print(count, m.analyze('key').tonic, [n.beat for n in m.notesAndRests])
        count+=1            

NEW PART
Horn 2
0 B- [1.0]
1 F [0.5, 0.5] [0.5, 0.5] [0.5, 0.5] [0.5, 0.5]
2 F [0.25, 0.25, 0.25, 0.25] [0.5, 0.5] [1.0] [1.0]
3 F [0.5, 0.25, 0.25] [0.5, 0.25, 0.25] [0.5, 0.5] [0.5, 0.5]
4 F [0.5, 0.25, 0.25] [1.0] [1.0] [1.0]
5 F [0.5, 0.5] [0.5, 0.5] [0.5, 0.5] [0.5, 0.5]
6 F [0.5, 0.25, 0.25] [0.5, 0.25, 0.25] [1.0] [1.0]
7 F [0.5, 0.5] [0.5, 0.5] [0.5, 0.5] [0.5, 0.5]
8 F [0.25, 0.25, 0.25, 0.25] [0.5, 0.5] [1.0] [1.0]
9 F [0.5, 0.25, 0.25] [0.5, 0.25, 0.25] [0.5, 0.5] [0.5, 0.5]
10 F [0.5, 0.25, 0.25] [1.0] [1.0] [1.0]
11 F [0.5, 0.5] [0.5, 0.5] [0.5, 0.5] [0.5, 0.5]
12 F [0.5, 0.25, 0.25] [0.5, 0.25, 0.25] [1.0] [1.0]
13 F [0.5, 0.25, 0.25] [0.25, 0.25, 0.25, 0.25] [2.0]
14 D [0.25, 0.25, 0.25, 0.25] [1.0] [2.0]
15 F [0.5, 0.5] [0.5, 0.5] [0.5, 0.25, 0.25] [0.5, 0.25, 0.25]
16 F [0.5, 0.5] [0.5, 0.5] [1.5] [0.5]
17 F [0.5, 0.25, 0.25] [1.0] [0.5, 0.5] [0.5, 0.25, 0.25]
18 F [2.0] [0.25, 0.25, 0.25, 0.25] [1.0]
19 F [0.5, 1.0] [0.5, 0.5] [1.0] [0.5]
20 F [0.5, 0.5] [0.5, 0.25, 0

In [34]:
part_names = {}
for piece in tqdm_notebook(allBach):
    score = piece.parse()
    for p in score.parts:
        name = p.partName
        if name not in part_names:
            part_names[name] = 1
            print(name)
        else:
            part_names[name] += 1
    

In [35]:
part_names

{None: 161,
 'Horn 2': 11,
 'Soprano': 405,
 'Alto': 407,
 'Tenor': 407,
 'Bass': 407,
 'Cornet 1': 2,
 'Cornet 2': 2,
 'Oboe': 3,
 'Trumpet 1': 10,
 'Trumpet 2': 10,
 'Trumpet 3': 10,
 'Timpani': 16,
 'Continuo': 10,
 'Horn 1': 10,
 'Horn 3': 2,
 'Violin': 5,
 'Recorders': 1,
 'Soprano\rOboe 1,2\rViolin1': 1,
 'Alto\rViloin 2': 1,
 'Tenor\rViola': 1,
 'Bass\rContinuo': 1,
 'Recorder 1': 1,
 'Recorder 2': 1,
 'Recorder 3': 1,
 'Oboe 1': 3,
 'Oboe 2': 3,
 'Oboe 3': 2,
 'Violin 1': 6,
 'Violin 2': 6,
 'Viola': 6,
 'Soprano 1': 2,
 'Soprano 2': 2,
 'Flutes': 1,
 "Oboe d'amore": 1,
 'Oboe da caccia': 1,
 'Organ': 2,
 'Trumpet 1,2': 1,
 'Soprano Oboe 1 Violin1': 1,
 'Alto Oboe 2 Viloin 2': 1,
 'Tenor Viola': 1,
 'Bass Continuo': 1,
 'Piano': 4,
 'MusicXML Part': 2}